<a href="https://colab.research.google.com/github/Levan-Danelia/FRTB/blob/main/FRTB_CRDL_Securitization_Non_ACTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Setup and Initial Data
# Import necessary libraries
import pandas as pd
import numpy as np

# --- Initial Portfolio Data ---
# This data represents the starting positions and their sensitivities.
data = [
    {'Position ID': 1, 'Bucket': 14, 'Maturity': '10Y', 'Sensitivity': 6205554},
    {'Position ID': 2, 'Bucket': 14, 'Maturity': '5Y', 'Sensitivity': 7433026},
    {'Position ID': 3, 'Bucket': 4, 'Maturity': '1Y', 'Sensitivity': 3624929},
    {'Position ID': 4, 'Bucket': 4, 'Maturity': '3Y', 'Sensitivity': 121878},
]

# --- Regulatory Parameters ---
# Risk Weights for CSR Securitizations non-ACTP as per Article 325am, Table 7
risk_weights = {
    14: 0.015,  # ABS - Credit Cards (Non-senior IG)
    4: 0.020,   # CMBS (Senior IG)
}

# Create an initial DataFrame
portfolio_df = pd.DataFrame(data)

print("--- Initial Setup Complete ---")
print(f"Loaded {len(portfolio_df)} positions.")

--- Initial Setup Complete ---
Loaded 4 positions.


In [2]:
# Cell 2: Step 1 & 2 - Identify Risk Factors and Define Sensitivities
# For this portfolio, each position has a unique risk factor (bucket + tenor).
# The sensitivities are provided directly in the initial data.

print("### Step 1 & 2: Identify Risk Factors and Sensitivities ###")
print("The initial portfolio with sensitivities for each position:")

# Display the initial portfolio DataFrame
portfolio_df.set_index('Position ID', inplace=True)
display(portfolio_df)

### Step 1 & 2: Identify Risk Factors and Sensitivities ###
The initial portfolio with sensitivities for each position:


,Bucket,Maturity,Sensitivity
Position ID,,,
1,14,10Y,6205554
2,14,5Y,7433026
3,4,1Y,3624929
4,4,3Y,121878


In [3]:
# Cell 3: Step 3 - Calculate Net Sensitivities (sk)
# As per Article 325f, paragraph 5, we net sensitivities for identical risk factors.
# In this specific portfolio, each position is a unique risk factor, so there is no netting.

print("\n### Step 3: Calculate Net Sensitivities (sk) ###")
print("For this portfolio, net sensitivities are equal to the individual sensitivities.")

# The original sensitivities are already the net sensitivities for this case
net_sensitivities_df = portfolio_df.copy()
net_sensitivities_df['Net_Sensitivity_sk'] = net_sensitivities_df['Sensitivity']
display(net_sensitivities_df[['Bucket', 'Maturity', 'Net_Sensitivity_sk']])


### Step 3: Calculate Net Sensitivities (sk) ###
For this portfolio, net sensitivities are equal to the individual sensitivities.


,Bucket,Maturity,Net_Sensitivity_sk
Position ID,,,
1,14,10Y,6205554
2,14,5Y,7433026
3,4,1Y,3624929
4,4,3Y,121878


In [4]:
# Cell 4: Step 4 - Calculate Weighted Sensitivities (WSk)
# Each net sensitivity is multiplied by its regulatory risk weight (RWk) from Article 325am.

print("\n### Step 4: Calculate Weighted Sensitivities (WSk) ###")
print("Applying regulatory risk weights to the net sensitivities.")

# Map the risk weights to the DataFrame
net_sensitivities_df['Risk_Weight_RWk'] = net_sensitivities_df['Bucket'].map(risk_weights)

# Calculate the weighted sensitivity
net_sensitivities_df['Weighted_Sensitivity_WSk'] = net_sensitivities_df['Net_Sensitivity_sk'] * net_sensitivities_df['Risk_Weight_RWk']

display(net_sensitivities_df[['Bucket', 'Maturity', 'Net_Sensitivity_sk', 'Risk_Weight_RWk', 'Weighted_Sensitivity_WSk']])


### Step 4: Calculate Weighted Sensitivities (WSk) ###
Applying regulatory risk weights to the net sensitivities.


,Bucket,Maturity,Net_Sensitivity_sk,Risk_Weight_RWk,Weighted_Sensitivity_WSk
Position ID,,,,,
1,14,10Y,6205554,0.015,93083.31
2,14,5Y,7433026,0.015,111495.39
3,4,1Y,3624929,0.020,72498.58
4,4,3Y,121878,0.020,2437.56


In [5]:
# Cell 5: Step 5 & 7 - Intra-Bucket Aggregation (Medium Scenario)
# This function calculates the bucket-specific capital (Kb) by aggregating
# the weighted sensitivities within each bucket using the intra-bucket correlation.

def calculate_kb(bucket_df, scenario='medium'):
    """
    Calculates the bucket-specific capital (Kb) based on Article 325f, paragraph 7.
    """
    # Determine intra-bucket correlation (rho_kl) as per Article 325an
    # Since maturities are different, rho_tenor is 80%. Other components are 1.0.
    rho_kl_medium = 0.80

    # Adjust correlation based on the scenario as per Article 325h
    if scenario == 'high':
        rho_kl = min(rho_kl_medium * 1.25, 1.0)
    elif scenario == 'low':
        rho_kl = max(2 * rho_kl_medium - 1, 0.75 * rho_kl_medium)
    else: # medium
        rho_kl = rho_kl_medium

    ws = bucket_df['Weighted_Sensitivity_WSk'].values

    # Sum of squares of weighted sensitivities
    sum_ws_sq = np.sum(ws**2)

    # Sum of cross-products
    cross_product_sum = 0
    if len(ws) > 1:
        # This loop calculates the sum of (rho_kl * WSk * WSl) for all pairs (k, l) where k != l
        for i in range(len(ws)):
            for j in range(len(ws)):
                if i != j:
                    cross_product_sum += rho_kl * ws[i] * ws[j]

    # The term inside the square root is floored at zero
    k_squared = max(sum_ws_sq + cross_product_sum, 0)

    return np.sqrt(k_squared)

print("\n### Step 5 & 7: Intra-Bucket Aggregation (Medium Scenario) ###")
print("Calculating bucket-specific capital (Kb) for the Medium Correlation Scenario.")

# Calculate Kb for each bucket
bucket_capital = {}
for bucket_id, group in net_sensitivities_df.groupby('Bucket'):
    bucket_capital[bucket_id] = calculate_kb(group, scenario='medium')

# Also calculate Sb for each bucket (sum of weighted sensitivities) for the next step
s_b_values = net_sensitivities_df.groupby('Bucket')['Weighted_Sensitivity_WSk'].sum().to_dict()

kb_results_df = pd.DataFrame({
    'Bucket': list(bucket_capital.keys()),
    'Kb_Medium': list(bucket_capital.values()),
    'Sb_Medium': list(s_b_values.values())
})

display(kb_results_df)


### Step 5 & 7: Intra-Bucket Aggregation (Medium Scenario) ###
Calculating bucket-specific capital (Kb) for the Medium Correlation Scenario.


,Bucket,Kb_Medium,Sb_Medium
0,4,74462.992302,74936.14
1,14,194167.712335,204578.70


In [6]:
# Cell 6: Step 6 & 8 - Across-Bucket Aggregation (Medium Scenario)
# This function aggregates the bucket-level capital (Kb) to get the final
# risk class capital for a given scenario.

def calculate_final_capital(kb_df, scenario='medium'):
    """
    Calculates the final risk class capital based on Article 325f, paragraph 8.
    """
    # For this risk class, gamma is 0.0 as per Article 325ao
    gamma_medium = 0.0

    # Adjust correlation based on the scenario as per Article 325h
    if scenario == 'high':
        gamma = min(gamma_medium * 1.25, 1.0)
    elif scenario == 'low':
        gamma = max(2 * gamma_medium - 1, 0.75 * gamma_medium)
    else: # medium
        gamma = gamma_medium

    k_values = kb_df[f'Kb_{scenario.capitalize()}'].values
    s_values = kb_df[f'Sb_{scenario.capitalize()}'].values

    # Sum of squares of Kb values
    sum_k_sq = np.sum(k_values**2)

    # Sum of cross-bucket terms
    cross_bucket_sum = 0
    if len(k_values) > 1:
        for i in range(len(s_values)):
            for j in range(len(s_values)):
                if i != j:
                    cross_bucket_sum += gamma * s_values[i] * s_values[j]

    # The term inside the square root is floored at zero
    final_capital_sq = max(sum_k_sq + cross_bucket_sum, 0)

    return np.sqrt(final_capital_sq)

print("\n### Step 6 & 8: Across-Bucket Aggregation (Medium Scenario) ###")
print("Aggregating bucket capital using cross-bucket correlations.")

# Calculate final capital for the medium scenario
medium_scenario_capital = calculate_final_capital(kb_results_df, scenario='medium')

print(f"\nMedium Scenario CSR Delta Capital Requirement: {medium_scenario_capital:,.2f}")


### Step 6 & 8: Across-Bucket Aggregation (Medium Scenario) ###
Aggregating bucket capital using cross-bucket correlations.

Medium Scenario CSR Delta Capital Requirement: 207,956.34


In [7]:
# Cell 7: Step 9 - Calculate High and Low Correlation Scenarios
# Per Article 325h, the calculation is repeated for high and low correlation scenarios.

print("\n### Step 9: Calculate High and Low Correlation Scenarios ###")

# --- High Correlation Scenario ---
bucket_capital_high = {}
for bucket_id, group in net_sensitivities_df.groupby('Bucket'):
    bucket_capital_high[bucket_id] = calculate_kb(group, scenario='high')
s_b_values_high = s_b_values # Sb values do not change with correlation scenario
kb_results_df['Kb_High'] = kb_results_df['Bucket'].map(bucket_capital_high)
kb_results_df['Sb_High'] = kb_results_df['Bucket'].map(s_b_values_high)
high_scenario_capital = calculate_final_capital(kb_results_df, scenario='high')

# --- Low Correlation Scenario ---
bucket_capital_low = {}
for bucket_id, group in net_sensitivities_df.groupby('Bucket'):
    bucket_capital_low[bucket_id] = calculate_kb(group, scenario='low')
s_b_values_low = s_b_values # Sb values do not change with correlation scenario
kb_results_df['Kb_Low'] = kb_results_df['Bucket'].map(bucket_capital_low)
kb_results_df['Sb_Low'] = kb_results_df['Bucket'].map(s_b_values_low)
low_scenario_capital = calculate_final_capital(kb_results_df, scenario='low')

# --- Display Summary ---
scenario_summary = pd.DataFrame([
    {'Scenario': 'Medium', 'Capital': medium_scenario_capital},
    {'Scenario': 'High', 'Capital': high_scenario_capital},
    {'Scenario': 'Low', 'Capital': low_scenario_capital}
])
display(scenario_summary.set_index('Scenario'))


### Step 9: Calculate High and Low Correlation Scenarios ###


,Capital
Scenario,
Medium,207956.336128
High,217871.222450
Low,197544.440317


In [8]:
# Cell 8: Step 10 - Final Charge Calculation
# The final own funds requirement is the maximum of the three scenarios.

print("\n### Step 10: Final Charge Calculation ###")

final_capital_charge = scenario_summary['Capital'].max()

print(f"\nFinal CSR Delta Capital Requirement: {final_capital_charge:,.2f}")

# Display the final summary card as a DataFrame for a clean output
summary_card = pd.DataFrame([{'Final CSR Delta Capital Requirement': f"{final_capital_charge:,.2f}"}])
display(summary_card)


### Step 10: Final Charge Calculation ###

Final CSR Delta Capital Requirement: 217,871.22


,Final CSR Delta Capital Requirement
0,"217,871.22"
